# Automatická detekcia TLE na snímkach AMOS pomocou hlbokého učenia

## Príprava prostredia

Príkazy na inštaláciu potrebných knižníc.

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install opencv-contrib-python
!pip install cv2

Import potrebných knižníc.

In [ ]:
import cv2
import os
import shutil
import csv

## Trénovanie modelu

Príkaz na začatie trénovania

In [ ]:
python yolov5/train.py --img 640 --rect --batch 60 --epochs 300
--data ./yolo/tle8.yaml --cfg ./yolov5/models/yolov5s.yaml
--save-period 10 --workers 0

## Iteratívny postup pridávania dát 

### Spojenie dát zo všetkých priečinkov do jedného 
reorg_dir: pôvodný priečinok s dalšími priečinkami <br>
target_dir: výstupný priečinok

In [ ]:
reorg_dir = "/home/jovyan/data/lightning/Petra/yolo/AMOS2016/2016"
target_dir = "/home/jovyan/data/lightning/Petra/yolo/AMOS2016/vsetko" 

for root, dirs, files in os.walk(reorg_dir):
    for name in files:
        subject = root + "/" + name
        n = 1
        name_orig = os.path.splitext(name)[0]
        extension = os.path.splitext(name)[1]
        while os.path.exists(target_dir + "/" + name):
            name = str(n)+"_" +name_orig + extension
            n = n + 1
        newfile = target_dir + "/" + name;
        shutil.move(subject, newfile)

## Výpočet ľavého krajného roku zo stredu


In [ ]:
x=0.161719
y=0.0557292
w=0.034375
h=0.046875
a=(x-(w/2))*1280
b=(y-(h/2))*960
c=(x+(w/2))*1280
d=(y+(h/2))*960
print(a,b,c,d)

## Vyčiernenie falošne pozitívnych javov, ktoré sa stále vyskytujú na snímke


In [ ]:
rootPath = r'/home/jovyan/data/lightning/Petra/yolo/AMOS2016/vsetko'
for root, dirs, files in os.walk(rootPath):
    for file in files:
        name = os.path.splitext(file)[0]
        extension = os.path.splitext(file)[1]
        if name[-5:] == "VAZEC":
            im = Image.open(os.path.join(root, "%s" % (file)))
            im1 = ImageDraw.Draw(im)
            im1.rectangle([(255.000192, 0), (340.000128, 43.000032)], outline="black",fill="black")
            im1.rectangle([(1211.99936, 435.99950400000006), (1279.99936, 520.999536)], outline="black",fill="black")
            im1.rectangle([(188.0, 387.00033600000006), (235.99999999999997, 467.00030399999997)], outline="black",fill="black")
            im1.rectangle([(94.99993599999999,223.00017599999998), (134.0,264.000144)], outline="black",fill="black")
            im1.rectangle([(233.00032000000002,874.9998240000001), (386.0 ,959.999856)], outline="black",fill="black")
            im.save(os.path.join(rootPath, "%s" % (file)))
        if name[-3:] == "AGO":
            im = Image.open(os.path.join(root, "%s" % (file)))
            im1 = ImageDraw.Draw(im)
            im1.rectangle([(236.99987200000004, 314.99952 ), (287.99980800000003,401.99952)], outline="black",fill="black")
            im.save(os.path.join(rootPath, "%s" % (file)))
        if name[-5:] == "Senec":
            im = Image.open(os.path.join(root, "%s" % (file)))
            im1 = ImageDraw.Draw(im)
            im1.rectangle([(1172.999808, 357.0), (1207.9998719999999, 393.0)], outline="black",fill="black")
            im1.rectangle([(1131.999552 ,747.0), (1178.999488, 807.0)], outline="black",fill="black")
            im1.rectangle([(164.999808,89.00015999999998), (287.999872, 216.99984)], outline="black",fill="black")
            im.save(os.path.join(rootPath, "%s" % (file)))
        if name[-3:] == "KNM":
            im = Image.open(os.path.join(root, "%s" % (file)))
            im1 = ImageDraw.Draw(im)
            im1.rectangle([(148.999488,2.9999520000000013 ), (185.999552,33.999984)], outline="black",fill="black")
            im1.rectangle([(185.00032000000002, 31.000032 ), (229.00032, 76.000032)], outline="black",fill="black")
            im.save(os.path.join(rootPath, "%s" % (file)))
        if name[-4:] == "ARBO":
            im = Image.open(os.path.join(root, "%s" % (file)))
            im1 = ImageDraw.Draw(im)
            im1.rectangle([(227.00032 ,111.00048), (275.00032, 168.00048)], outline="black",fill="black")
            im.save(os.path.join(rootPath, "%s" % (file)))

## Detekcia vtedy najlepším modelom na novom datasete

In [ ]:
!python yolov5/detect.py --weights ./yolov5/runs/train/exp70/weights/best.pt --source ./yolo/AMOS2018/vsetko --iou 0.3 --conf 0.45 --img 640 --save-txt 

## Odstránenie AGO-Spec snímok, ktoré neobsahujú TLE

In [ ]:
rootPath = '/home/jovyan/data/lightning/Petra/yolov5/runs/detect/exp366/labels'
for root, dirs, files in os.walk(rootPath):
    for file in files:
        name = os.path.splitext(file)[0]
        extension = os.path.splitext(file)[1]
        if name[-8:] == "AGO-Spec":
            full_file_path = os.path.join(rootPath, file)
            os.remove(full_file_path)
            

## Vytvorenie zoznamu len anotovaných snímok

In [ ]:
rootPath = r'/home/jovyan/data/lightning/Petra/yolov5/runs/detect/exp399/labels'

with open("anotovane399.txt", "w") as a:
    for path, subdirs, files in os.walk(rootPath):
        for filename in files:
            f = os.path.join(filename)
            name = os.path.splitext(f)[0]
            extension = os.path.splitext(f)[1]
            a.write(str(name)+".jpg"+ os.linesep)

## Odstránenie snímok na ktorých sa nenachádza anotácia

In [ ]:
path = '/home/jovyan/data/lightning/Petra/yolov5/runs/detect/exp399'
with open('anotovane399.txt',"r") as f:
    file_name = f.readlines()
    file_name = [x[:-1] for x in file_name]
    for filename in os.listdir(path):
        if filename == "labels":
            continue
        if filename not in file_name:
            full_file_path = os.path.join(path, filename)
            os.remove(full_file_path)
    

## Finálna detekcia

In [ ]:
!python yolov5/detect.py --weights
./yolov5/runs/train/exp103/weights/epoch190.pt
--source ./yolo/test3 --iou 0.3 --conf 0.45 --img 640 --save-txt

## Vytvorenie výstupnej tabuľky so všetkými nájdenými javmi počas práce
csvOut: názov výstupne tabuľky <br>
path: cesta ku všetkým nájdeným obrázkom s anotáciami

In [ ]:
csvOut='vystupfinal2.csv'
path = '/home/jovyan/data/lightning/Petra/final'
with open(csvOut, mode='w') as csv_file:
    fieldnames = ['filename','x', 'y', 'width', 'height']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for root, dirs, files in os.walk(path):
        for name in files:
            file = path+"/"+name
            txt = os.path.splitext(name)[1]
            if txt == ".txt":
                with open(file) as f:
                    name_orig = os.path.splitext(name)[0]
                    lines = f.readlines()
                    dlzka = len(lines)
                    i=0
                    while i<dlzka:
                        line = lines[i]
                        splitted = line.split(" ")
                        list(map(lambda x:x.strip(),splitted))
                        writer.writerow({'filename': name_orig,'x':splitted[1], 'y':splitted[2], 'width':splitted[3], 'height':splitted[4]})
                        i=i+1